In [ ]:
%load_ext sql

In [ ]:
from time import time
import configparser
import matplotlib.pyplot as plt
import pandas as pd
import boto3

# STEP 1: Get the params of the created redshift cluster 

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
KEY=config.get('AWS','KEY')
SECRET= config.get('AWS','SECRET')

DWH_DB= config.get("CLUSTER","DB_NAME")
DWH_DB_USER= config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD= config.get("CLUSTER","DB_PASSWORD")
DWH_PORT = config.get("CLUSTER","DB_PORT")

# STEP 2: Connect to the Redshift Cluster

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

# STEP 3: Create Staging Tables

In [ ]:
%%sql 
DROP TABLE IF EXISTS staging_songs;
CREATE TABLE "staging_songs" (
    "num_songs" INTEGER ,
    "artist_id" VARCHAR(60) NOT NULL,
    "artist_latitude" VARCHAR(50),
    "artist_longitude" VARCHAR(50),
    "artist_location" VARCHAR(200),
    "artist_name" VARCHAR(100),
    "song_id" VARCHAR(60) NOT NULL,
    "title" VARCHAR(200) NOT NULL,
    "duration" numeric(10,5) NOT NULL,
    "year" INTEGER NOT NULL
);

DROP TABLE IF EXISTS staging_events;
CREATE TABLE "staging_events" (
    "artist" VARCHAR(200),
    "auth" VARCHAR(12),
    "firstName" VARCHAR(30),
    "gender" VARCHAR(1),
    "itemInSession" SMALLINT,
    "lastName" VARCHAR(30),
    "length" numeric(10,5),
    "level" VARCHAR(6),
    "location" VARCHAR(300), 
    "method" VARCHAR(3),
    "page" VARCHAR(20),
    "registeration" DECIMAL,
    "sessionId" BIGINT,
    "song" VARCHAR(300),
    "status" VARCHAR(4),
    "ts" TIMESTAMP,
    "userAgent" VARCHAR(300),
    "userId" INTEGER

);

# STEP 4: Write ETL to copy data from JSON files in S3 to Redshift

In [ ]:
%%time
song_data = config.get('S3','SONG_DATA')
qry = """
    copy staging_songs from {}
    credentials 'aws_iam_role={}'
    format as json 'auto' compupdate off STATUPDATE ON TRUNCATECOLUMNS region 'us-west-2';
""".format(song_data, DWH_ROLE_ARN)

%sql $qry

In [ ]:
%%time
log_data = config.get('S3','LOG_DATA')
log_json_path = config.get('S3', 'LOG_JSONPATH')
qry = """
    copy staging_events from {}
    credentials 'aws_iam_role={}'
    format as json {} compupdate off TIMEFORMAT 'epochmillisecs' TRUNCATECOLUMNS region 'us-west-2';
""".format(log_data, DWH_ROLE_ARN, log_json_path)

%sql $qry

# STEP 4: Create Analytics Tables

In [ ]:
%%sql 
DROP TABLE IF EXISTS songplays;
DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS songs;
DROP TABLE IF EXISTS artists;
DROP TABLE IF EXISTS times;

CREATE TABLE songs 
(
    song_id VARCHAR PRIMARY KEY sortkey distkey,
    title VARCHAR,
    artist_id VARCHAR,
    year INTEGER,
    duration numeric(10,5)
);

CREATE TABLE artists 
(
    artist_id VARCHAR PRIMARY KEY sortkey,
    name VARCHAR,
    location VARCHAR, 
    lattitude VARCHAR,
    longitude VARCHAR
);

CREATE TABLE songplays 
(
    songplay_id INTEGER IDENTITY(0,1) PRIMARY KEY,
    start_time TIMESTAMP NOT NULL,
    user_id INTEGER NOT NULL,
    level VARCHAR,
    song_id VARCHAR distkey, 
    artist_id VARCHAR sortkey,
    session_id INTEGER,
    location VARCHAR,
    user_agent VARCHAR
);

CREATE TABLE users 
(
    user_id INTEGER PRIMARY KEY sortkey,
    first_name VARCHAR,
    last_name VARCHAR,
    gender VARCHAR,
    level VARCHAR
);

CREATE TABLE times 
(
    ts TIMESTAMP PRIMARY KEY sortkey,
    hour INTEGER,
    day INTEGER,
    week INTEGER,
    month INTEGER,
    year INTEGER,
    weekday INTEGER
);

# STEP 6: Insert from staging tables into analytics tables

In [ ]:
%%time

qry = """
    INSERT INTO times
    SELECT 
        e.ts,
        EXTRACT(hour from e.ts) as hour,
        EXTRACT(day from e.ts) as day,
        EXTRACT(week from e.ts) as week,
        EXTRACT(month from e.ts) as month,
        EXTRACT(year from e.ts) as year,
        EXTRACT(weekday from e.ts) as weekday FROM staging_events e
     """

%sql $qry

In [ ]:
%%time

qry = """
    INSERT INTO users
    SELECT 
        userId,
        firstName,
        lastName,
        gender,
        level FROM staging_events where userId is not NULL"""

%sql $qry

In [ ]:
%%time

qry = """
    INSERT INTO artists
    SELECT 
        artist_id,
        artist_name,
        artist_location,
        artist_latitude,
        artist_longitude FROM staging_songs where artist_id is not null """

%sql $qry

In [ ]:
%%time

qry = """
    INSERT INTO songs
    SELECT 
        song_id,
        title,
        artist_id,
        year,
        duration FROM staging_songs where song_id is not null """

%sql $qry

In [ ]:
%%time

qry = """
    INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
    SELECT 
        ts,
        userId,
        level,
        song_id,
        artist_id,
        sessionId,
        location,
        userAgent
        FROM staging_songs s JOIN staging_events e
        ON s.artist_name = e.artist 
        AND s.title = e.song """

%sql $qry